In [1]:
!pip install --quiet transformers datasets sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 29.8 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset_builder
from datasets import load_dataset
import transformers
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

In [3]:
ds_builder = load_dataset_builder("xsum")
print(ds_builder.info.description)
print(ds_builder.info.features)


Extreme Summarization (XSum) Dataset.

There are three features:
  - document: Input news article.
  - summary: One sentence summary of the article.
  - id: BBC ID of the article.


{'document': Value(dtype='string', id=None), 'summary': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None)}


In [4]:
dataset = load_dataset("xsum")
dataset

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})

In [5]:
for k, v in dataset["train"][0].items():
    print(k,'=',v)
    print()

document = The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.
Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.
Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct.
Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.
First Minister Nicola Sturgeon visited the area to inspect the damage.
The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.
Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.
However, she said more preventative work could have been carried out to ensure the retaining wall did not fail.
"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - b

In [6]:
model_name = "google/pegasus-xsum"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
model.eval()

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(512, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_nor

In [7]:
model.config

PegasusConfig {
  "_name_or_path": "google/pegasus-xsum",
  "activation_dropout": 0.1,
  "activation_function": "relu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "PegasusForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 16,
  "decoder_start_token_id": 0,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 16,
  "eos_token_id": 1,
  "extra_pos_embeddings": 0,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "label2id": {
    "LABEL_0": 0,


In [ ]:
input_example = dataset["train"][:2]["document"]
batch_example = tokenizer(input_example, truncation=True, padding="longest", return_tensors="pt").to(device)

summarized_example = model.generate(**batch_example)
summary_example = tokenizer.batch_decode(summarized_example, skip_special_tokens=True)
summary_example

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['A clean-up operation is under way in parts of Dumfries and Galloway hit by flooding over the weekend.',
 'Two tourist buses have been destroyed in a suspected arson attack in Londonderry.']

In [ ]:
# A way to get encoder output [but then requires more than 1 pass overall]
input_ids = tokenizer(input_example, padding=True, return_tensors="pt").input_ids.to(device)
encoder_hidden_state_method_1 = model.base_model.encoder(input_ids, return_dict=True).last_hidden_state   # [batch_size, seq_length, dims]

In [ ]:
# This gives both encoder and decoder hidden states
batch = tokenizer(input_example, truncation=True, padding="longest", return_tensors="pt").to(device)
outputs = model.generate(**batch, output_hidden_states=True, output_scores=True, return_dict_in_generate=True)

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (64) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
outputs.sequences.shape     # 1 more than number of generated tokens (see https://github.com/huggingface/transformers/issues/21374#issuecomment-1409041676)

torch.Size([2, 23])

In [ ]:
tokenizer.batch_decode(outputs.sequences, skip_special_tokens=True)

['A clean-up operation is under way in parts of Dumfries and Galloway hit by flooding over the weekend.',
 'Two tourist buses have been destroyed in a suspected arson attack in Londonderry.']

In [ ]:
print("0-indexed encoder last layer is:", torch.argmin(
    torch.Tensor([abs(torch.sum(encoder_hidden_state_method_1 - outputs.encoder_hidden_states[i]).item()) for i in range(len(outputs.encoder_hidden_states))])
).item())
encoder_last_hidden_state = outputs.encoder_hidden_states[-1]

0-indexed encoder last layer is: 16


In [ ]:
[abs(torch.sum(encoder_hidden_state_method_1 - outputs.encoder_hidden_states[i]).item()) for i in range(len(outputs.encoder_hidden_states))]

[71682.875,
 319305.75,
 380437.875,
 372767.0625,
 379040.3125,
 396388.46875,
 389333.625,
 405380.75,
 481518.21875,
 537423.375,
 683545.875,
 929661.75,
 1105926.0,
 1460250.25,
 1658042.5,
 1867806.625,
 1994.103515625]

In [ ]:
outputs.decoder_hidden_states[0][0].shape      # [29,16+1,16,1,1024]

torch.Size([16, 1, 1024])

In [ ]:
type(outputs)

transformers.generation.utils.BeamSearchEncoderDecoderOutput

In [ ]:
# https://github.com/huggingface/transformers/issues/21374

def extract_decoder_hidden_states(
    generate_output_dict,
    hidden_layer_idx=-1,
):
    """
    Extracts the decoder hidden states representation from
    GreedySearchEncoderDecoderOutput and BeamSearchEncoderDecoderOutput,
    associated with the `sequences` output.
    - generate_output_dict: output dict from the model.generate() method
      you should add the following arguments to generate:
        - output_hidden_states=True
        - output_scores=True
        - return_dict_in_generate=True
    - hidden_layer_idx: index of the layer to extract the representation from (-1 == last one)
    """
    greedy = isinstance(generate_output_dict, transformers.generation.utils.GreedySearchEncoderDecoderOutput)
    beamy  = isinstance(generate_output_dict, transformers.generation.utils.BeamSearchEncoderDecoderOutput)

    if greedy:
        # in greedy decoding, the beam_indices is not present, so we create one
        # where the first beam is always selected
        scores = generate_output_dict['scores']
        device = generate_output_dict['sequences'].device
        beam_indices = torch.arange(scores[0].shape[0]).view(-1, 1)
        beam_indices = beam_indices.expand(-1, len(scores)).to(device)
    elif beamy:
        if 'beam_indices' not in generate_output_dict:
            raise RuntimeError(
                "You should export the scores with output_scores=True when " \
                "calling extract_decoder_hidden_states with " \
                "BeamSearchEncoderDecoderOutput"
            )
        beam_indices = generate_output_dict['beam_indices'].clone()
    else:
        raise NotImplementedError(
            "extract_decoder_hidden_states only works with " \
            "GreedySearchEncoderDecoderOutput and BeamSearchEncoderDecoderOutput " \
            "output types."
        )

    # handling of the target length and preparing the masking for tokens
    # outside of that length
    beam_indices_mask = beam_indices < 0
    max_beam_length = (1 - beam_indices_mask.long()).sum(-1).max()
    beam_indices = beam_indices[:, :max_beam_length]
    beam_indices_mask = beam_indices_mask[:, :max_beam_length]
    beam_indices[beam_indices_mask] = 0

    seqlen = generate_output_dict['sequences'].shape[1] - 1

    # creating the output hidden_states representation in format:
    # [bsz * beam_width ; seqlen ; featdim]
    decoder_hidden_states = torch.stack([
        generate_output_dict['decoder_hidden_states'][i][hidden_layer_idx][:,0,:].index_select(     # [:,0,:] is to get the first sequence
                                                                                                    # e.g., "1" in above [29,17,16,1,1024]
            dim=0,
            index=beam_indices[:,i] # reordering using the beam_indices
        )
        for i in range(seqlen)
    ]).transpose(0, 1)

    # setting to 0 the hidden_states were it doesn't make sense to have an output
    decoder_hidden_states[beam_indices_mask] = 0

    return decoder_hidden_states

In [ ]:
dec_hidden_states = extract_decoder_hidden_states(outputs,)

In [ ]:
dec_hidden_states.shape

torch.Size([2, 22, 1024])

In [ ]:
outputs.beam_indices

tensor([[ 0,  3,  6,  2,  2,  1,  0,  1,  1,  1,  2,  1,  3,  0,  0,  5,  3,  3,
          7,  3,  0,  0, -1],
        [ 8,  8, 12, 11,  8,  8,  8,  8,  8,  8,  8,  8,  8, 11, 10, -1, -1, -1,
         -1, -1, -1, -1, -1]], device='cuda:0')

In [ ]:
batch_size = 4
dataset_inference = dataset["test"]
all_encoder_last_hidden_states = []
all_decoder_last_hidden_states = []

In [ ]:
for i in range(0, len(dataset_inference), batch_size):
    batch = tokenizer(dataset_inference[i:min(i+batch_size,len(dataset_inference))]["document"], truncation=True, padding="longest", return_tensors="pt").to(device)
    outputs = model.generate(**batch, output_hidden_states=True, output_scores=True, return_dict_in_generate=True)
    all_encoder_last_hidden_states.append(outputs.encoder_hidden_states[-1])
    all_decoder_last_hidden_states.append(extract_decoder_hidden_states(outputs))

In [ ]:
ds = ds.with_format("torch", device=device)